### 문제 정의: KNeighborsClassifier 사용하여 타이타닉 생존자(1),사망자(0) 예측

기본 라이브러리 불러오기

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
# 한글깨짐방지

import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


[step1]데이터 준비_Seaborn에서 제공하는 titanic데이터셋 가져오기

In [ ]:
# load_dataset 함수를 사용하여 데이터프레임으로 변환
df = sns.load_dataset('titanic')

[step2]데이터 탐색 및 데이터 전처리

In [ ]:
df.head(3)

In [ ]:
# 데이터 자료형 확인
df.info()

In [ ]:
# NaN값이 많은 deck(객실 데크 위치) 열을 삭제
# embarked와 내용이 겹치는 embark_town(승선 도시) 열을 삭제

rdf = df.drop(['deck','embark_town'],axis=1)
rdf.info()

In [ ]:
# age열에 나이 데이터가 없는 모든 행을 삭제 -> 177개 NaN 삭제
rdf = rdf.dropna(subset=['age'], how='any') # age열에 있는 모든 NaN값을 지워
rdf.info()

In [ ]:
# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값 확인 -> 탑승한 도시의 첫글자
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
most_freq

'S'

In [ ]:
# 승객이 제일 많이 탄 도시가 'S'임을 다시 확인
rdf.describe(include='all')

In [ ]:
# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
rdf['embarked'].fillna(most_freq, inplace=True)
rdf.info()

[step3]분석에 사용할 속성을 선택

In [ ]:
# 분석에 활용할 열(속성)을 선택(생존여부, 객실등급, 성별, 나이, 타이타닉 탑승한 형제 자매수,
# 타이타닉에 탑승한 부모/자녀 수, 탑승한 곳(항구), C=Cherbourg, Q=Queenstown, S=Southhamton)
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
ndf.head()

In [ ]:
# 원학인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변경
# male,female -> [1,0],[0,1]

onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex],axis=1)
ndf.head(3)

In [ ]:
# embarked C, Q, S -> [1,0,0],[0,1,0],[0,0,1]

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked],axis=1)
ndf.head(3)

In [ ]:
# 기존 컬럼 삭제
ndf.drop(['sex','embarked'],axis=1,inplace=True)
ndf.head(3)

[step4]데이터셋 구분_훈련용(train data)/검증용(test data)

In [ ]:
# ndf -> X,y(문제집, 정답지)
X = ndf[['pclass','age','sibsp','parch','female','male','town_C','town_Q','town_S']] # 독립변수
y = ndf['survived']

X,y

In [ ]:
# train 데이터와 test 데이터로 구분(7:3 비율)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)
print(X_train.shape)
print(X_test.shape)

(499, 9)
(215, 9)


[step5] KNN 분류 모형_sklearn 사용

In [ ]:
# sklearn 라이브러리에서 KNN분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성(k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=5)

# train 데이터를 사용하여 모형 학습
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
# test 데이터를 가지고 y_hat을 예측(분류)
y_hat = knn.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

[0 1 0 1 1 0 0 0 0 0]
[0 1 0 1 0 0 1 0 0 0]


모델 성능 평가

In [ ]:
from sklearn import metrics

# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, y_hat)
print('테스트 평가지표 n_neighbors=5')
print(knn_report)

In [ ]:
# sklearn을 이용하면 전부 계산해준다.
print('accuracy:',metrics.accuracy_score(y_test, y_hat))
print('precision:',metrics.precision_score(y_test, y_hat))
print('recall:',metrics.recall_score(y_test, y_hat))
print('f1:',metrics.f1_score(y_test, y_hat))

accuracy: 0.7906976744186046
precision: 0.782051282051282
recall: 0.6853932584269663
f1: 0.7305389221556887


In [ ]:
# 이웃의 수에 따른 정확도 저장할 리스트 변수
train_accuracy = []
test_accuracy = []

# 1에서 10까지 n_neighbors를 적용
neighbors_settings = range(1,11)

for n_neighbors in neighbors_settings :
  # 모델 생성
  clf = KNeighborsClassifier(n_neighbors=n_neighbors)
  clf.fit(X_train, y_train)

  # 훈련 세트 정확도 저장
  train_accuracy.append(clf.score(X_train, y_train))

  # 테스트 세트 정확도 저장
  test_accuracy.append(clf.score(X_test, y_test))

plt.figure(dpi=100)
plt.rc('font', family='NanumBarunGothic')
plt.style.use('ggplot')
plt.plot(neighbors_settings, train_accuracy, label='훈련 정확도')
plt.plot(neighbors_settings, test_accuracy, label='테스트 정확도')
plt.ylabel('정확도')
plt.xlabel('n_neighbors')
plt.legend()
plt.show()

# n_neigbors=5인경우가 최상의 정확도
# 훈련정확도가 1.0이면 overfitting(과대적합)을 의심할 것
# 모델들의 기본 성능은 50%이상으로 생각할 것(이하면 모델을 다시 생각해볼 것)